In [6]:
import os

os.chdir("..")

from datetime import datetime
from pathlib import Path

import googlemaps

from pipelines.settings import settings

In [3]:
captions = {}
for file in Path(settings.BASE_DIR / "data/meronbangbidet").glob("*.txt"):
    with open(file) as fh:
        captions[file.stem] = fh.read()

In [4]:
key = list(captions.keys())[0]
entry_full = list(captions.values())[0]
entry = entry_full.split("\n")
key, entry

('2019-02-04_15-25-03_UTC',
 ['Walang Bidet sa The Peak, Grand Hyatt. *sa sobrang lasing ko hindi ko sinunod sarili kong layout :(*',
  'Water Pressure: -',
  'Cleanliness: ⭐️⭐️⭐️⭐️⭐️',
  'Poopability: -',
  'Weewee ka lang dito. :( well attended yung banyo. Solid doors. Hindi ka mapapasok ng kung sino. Pero hindi ka makakajebs.',
  '#meronbangbidet #walangbidet #hindipoopable',
  ''])

In [5]:
def get_stars(attr: str):
    return (
        next(
            (e for i, e in enumerate(entry) if e.lower().startswith(attr)),
            "",
        )
        .split(": ")[-1]
        .strip()
        .count("⭐")
    )


def get_caption():
    end_idx = next(
        (i for i in range(len(entry)) if "water pressure" in entry[i].lower()),
        -1,
    )
    start_idx = (
        next(
            (i for i in range(len(entry)) if "poopability" in entry[i].lower()),
            -1,
        )
        + 1
    )
    return "\n".join(
        [
            "\n".join(e for e in entry[:end_idx] if not e.startswith("#")),
            "\n".join(e for e in entry[start_idx:] if not e.startswith("#")),
        ]
    )


def get_timestamp():
    return datetime.strptime(key, "%Y-%m-%d_%H-%M-%S_%Z")


def get_has_bidet():
    return entry[0].lower().startswith("meron")


def get_location_name():
    return entry[0].lower().split(" sa ")[-1].split(".")[0]


def get_meta():
    return {
        "created": get_timestamp(),
        "has_bidet": get_has_bidet(),
        "establishment_name": get_location_name(),
        "content": get_caption(),
        "rating_cleanliness": get_stars("cleanliness"),
        "rating_water_pressure": get_stars("water pressure"),
        "rating_poopability": get_stars("poopability"),
    }


get_meta()

{'created': datetime.datetime(2019, 2, 4, 15, 25, 3),
 'has_bidet': False,
 'establishment_name': 'the peak, grand hyatt',
 'content': 'Walang Bidet sa The Peak, Grand Hyatt. *sa sobrang lasing ko hindi ko sinunod sarili kong layout :(*\nWeewee ka lang dito. :( well attended yung banyo. Solid doors. Hindi ka mapapasok ng kung sino. Pero hindi ka makakajebs.\n',
 'rating_cleanliness': 5,
 'rating_water_pressure': 0,
 'rating_poopability': 0}

In [8]:
gmaps = googlemaps.Client(key=settings.GOOGLE_MAPS_API_KEY)

In [19]:
# res = gmaps.geocode('grand hyatt manila', region='ph')
res = gmaps.places("the peak, grand hyatt", region="ph")
res

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'formatted_address': '60th Floor, Grand Hyatt Manila, 8th Avenue corner, 35th St, Taguig, 1634 Metro Manila',
   'geometry': {'location': {'lat': 14.5563657, 'lng': 121.0517549},
    'viewport': {'northeast': {'lat': 14.55761567989272,
      'lng': 121.0530695798927},
     'southwest': {'lat': 14.55491602010728, 'lng': 121.0503699201073}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png',
   'icon_background_color': '#FF9E67',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet',
   'name': 'The Peak Grill',
   'opening_hours': {'open_now': True},
   'photos': [{'height': 4032,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/102017190541913289234">Marvenn Chua</a>'],
     'photo_reference': 'AdCG2DP4c-pmV10s1olo2qWig8E_mA8Air22tfNKOMZ0Ku5-UC-ISQX8ebRMNe-QIJ71quSzhqgjEfl1BMNVfQyFLHXFzURQm2Zs34OHvdz3

In [21]:
res["results"][0]["geometry"]["location"]

{'lat': 14.5563657, 'lng': 121.0517549}